In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path='usage';
--select date, count(*) from usage_basic_kpi_fact_v7 where date >= '2020-06-28' and granularity = 'daily' and country_code = 'MO' and device_code = 'android-phone' group by date order by date;
--SELECT COUNT(country_code) FROM (SELECT DISTINCT country_code FROM usage_basic_kpi_fact_v7 WHERE date = '2020-07-19' AND granularity = 'daily' AND device_code IN ('android-phone', 'android-tablet')) country_tbl;
--select date, granularity, device_code, est_installs, est_install_base, est_usage_penetration / est_install_penetration, est_open_rate from usage_basic_kpi_fact_v6 where date >= '2020-09-05' limit 10;
--select date, granularity, device_code, count(*) from usage_basic_kpi_fact_v6 where date >='2020-08-29' and device_code like 'android%' group by date, granularity, device_code order by granularity, date, device_code;
--select est_average_active_users, country_code, est_population from usage_basic_kpi_fact_v1 where date = '2020-09-01' and device_code like 'ios%' and granularity = 'daily' and country_code = 'CN' limit 10;
--select * from usage_basic_kpi_fact_v1 where app_id = 711923939 and date in ('2019-06-01', '2019-06-08', '2019-06-15') and device_code like 'ios%' and granularity = 'daily' and country_code = 'CN' limit 10;
\x
select 
    device_code,
    sum(au) / sum(pop) as up, 
    sum(atd) / sum(au) as aad, 
    sum(tt) / sum(au) as atu, 
    sum(tv) / sum(au) as afu, 
    sum(tsd) / sum(au) as asd, 
    sum(tt) / sum(tv) as asd_1,
    sum(ib) / sum(pop) as ip,
    sum(tt) as tt,
    sum(tb) / sum(tv) / 1000 / 1000 as abps,
    sum(tb) / sum(au) / 1000 / 1000 as abpu,
    sum(atd) / sum(au) / 7 as atd_w
    --sum(tv) / sum(tsmc) as smc,
    --sum(tt) / sum(ttmc) as tmc
from (
    select 
        date, 
        country_code,
        device_code,
        app_id,
        est_average_session_duration * est_average_active_users / 60 / 1000 as tsd,
        est_population as pop,
        est_average_session_per_user * est_average_active_users as tv,
        est_total_time as tt, 
        est_average_active_days * est_average_active_users as ATD, 
        est_average_active_users AS au,
        est_installs as ib,
        est_average_bytes_per_session * est_average_active_users * est_average_session_per_user as tb
        --est_average_active_users * est_average_session_per_user / est_share_of_category_session as tsmc,
        --est_total_time / 60 / 1000 / est_share_of_category_time as ttmc
        from (
            select * from usage_basic_kpi_fact_v1 
            where 
            app_id in (284882215, 20600000009072, 20600003947760) 
            and date between '2018-10-01' and '2018-11-30' 
            and granularity = 'monthly' 
            --and country_code in ('RU', 'US', 'JP', 'IT')
            and country_code in ('US')
            ) tmp1 
) tmp2 group by device_code;
EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path='usage';

select
    device_code,
    sum(au) / sum(pop) as up, 
    sum(atd) / sum(au) as aad, 
    sum(tt) / sum(au) as atu, 
    sum(tv) / sum(au) as afu, 
    sum(tsd) / sum(au) as asd, 
    sum(tt) / sum(tv) as asd_1,
    sum(ib) / sum(pop) as ip,
    sum(tt) as tt,
    sum(tb) / sum(tv) / 1024 / 1024 as abps,
    sum(tb) / sum(au) / 1024 / 1024 as abpu,
    sum(atd) / sum(au) / 7 as atd_w
    --sum(tv) / sum(tsmc) as smc,
    --sum(tt) / sum(ttmc) as tmc
from (
    select 
        date, 
        country_code,
        device_code,
        app_id,
        est_average_session_duration * est_average_active_users / 60 / 1000 as tsd,
        est_population as pop,
        est_average_session_per_user * est_average_active_users as tv,
        est_total_time as tt, 
        est_average_active_days * est_average_active_users as ATD, 
        est_average_active_users AS au,
        est_install_base as ib,
        est_average_bytes_per_session * est_average_active_users * est_average_session_per_user as tb
        --est_average_active_users * est_average_session_per_user / est_share_of_category_session as tsmc,
        --est_total_time / 60 / 1000 / est_share_of_category_time as ttmc
        from (
            select * from usage_basic_kpi_fact_v1 
            where 
            app_id in (284882215, 20600000009072, 1393838612, 20600003947760) 
            and date between '2020-07-06' and '2020-10-17' 
            and granularity = 'daily' 
            --and country_code in ('RU', 'US', 'JP', 'IT')
            and country_code in ('US')
            ) tmp1
) tmp2
group by device_code
;
EOF


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path='usage';

select 
        date, 
        country_code,
        device_code,
        app_id,
        est_average_session_duration * est_average_active_users / 60 / 1000 as tsd,
        est_population as pop,
        est_average_session_per_user * est_average_active_users as tv,
        est_total_time as tt, 
        est_average_active_days * est_average_active_users as ATD, 
        est_average_active_users AS au,
        est_installs as ib,
        est_average_session_per_user as afu,
        est_average_bytes_per_session as mbps,
        est_average_bytes_per_user as mbpu,
        est_average_bytes_per_session * est_average_active_users * est_average_session_per_user as tb
        --est_average_active_users * est_average_session_per_user / est_share_of_category_session as tsmc,
        --est_total_time / 60 / 1000 / est_share_of_category_time as ttmc
        from (
            select * from usage_basic_kpi_fact_v1 
            where 
            app_id in (20600005513979, 1033282601) 
            and date between '2019-02-10' and '2019-02-16' 
            and granularity = 'weekly' 
            --and country_code in ('RU', 'US', 'JP', 'IT')
            and country_code in ('RU')
            ) tmp1 order by date desc;
EOF
            

In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/month=2015-02/device_id=1001/store_id=5/
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity=monthly/

In [0]:

rention_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/"
usage_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=monthly/"
# spark.read.format("parquet").load(rention_path).filter("date between '2019-02-01' and '2019-08-31'").createOrReplaceTempView("test_retention")
# spark.read.format("delta").load(usage_path).filter("date between '2019-02-01' and '2019-08-31'").createOrReplaceTempView("test_usage")
# spark.sql("select * from test_retention limit 1").show(1, False)
# spark.sql("select * from test_usage limit 1").show(1, False)
# spark.sql("select distinct kpi from test_retention ").show(100, False)

# spark.sql("select app_id, date, sum(estimate) as estimate from test_retention where app_id = 20600004251992 and date between '2019-02-01' and '2019-08-31'and kpi=301 and store_id=10 group by app_id, date").createOrReplaceTempView("retention_tmp")
# spark.sql("select app_id, date, sum(est_average_active_users) as au from test_usage where app_id = 20600004251992 and date between '2019-02-01' and '2019-08-31' and country_code  in ('US') group by app_id, date").createOrReplaceTempView("usage_tmp")


spark.sql("""select sum(test_retention.estimate*test_usage.est_average_active_users) as total_retention_users, sum(test_usage.est_average_active_users) as sum_au from test_retention  
left join test_usage 
on test_retention.app_id = test_usage.app_id 
and test_retention.date = test_usage.date 
where test_retention.app_id = 20600004251992 
and test_retention.date between '2019-06-01' and '2019-08-31' 
and test_retention.kpi=307 
and test_usage.country_code='US' 
and test_retention.store_id=10 
and test_retention.gender = 0 """).createOrReplaceTempView('total_retention_users')

spark.sql("select * from total_retention_users limit 10").show(10, False)


In [0]:

rention_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/"
usage_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=monthly/"
# spark.read.format("parquet").load(rention_path).filter("date between '2019-02-01' and '2019-08-31'").createOrReplaceTempView("test_retention")
# spark.read.format("delta").load(usage_path).filter("date between '2019-02-01' and '2019-08-31'").createOrReplaceTempView("test_usage")
# spark.sql("select * from test_retention limit 1").show(1, False)
# spark.sql("select * from test_usage limit 1").show(1, False)
# spark.sql("select distinct kpi from test_retention ").show(100, False)

# spark.sql("select app_id, date, sum(estimate) as estimate from test_retention where app_id = 20600004251992 and date between '2019-02-01' and '2019-08-31'and kpi=301 and store_id=10 group by app_id, date").createOrReplaceTempView("retention_tmp")
# spark.sql("select app_id, date, sum(est_average_active_users) as au from test_usage where app_id = 20600004251992 and date between '2019-02-01' and '2019-08-31' and country_code  in ('US') group by app_id, date").createOrReplaceTempView("usage_tmp")


spark.sql("""select test_retention.date, test_retention.estimate, test_usage.est_average_active_users from test_retention  
left join test_usage 
on test_retention.app_id = test_usage.app_id 
and test_retention.date = test_usage.date 
where test_retention.app_id = 20600004251992 
and test_retention.date between '2019-02-01' and '2019-08-31' 
and test_retention.kpi=307 
and test_usage.country_code='US' 
and test_retention.store_id=10 
and test_retention.gender = 0""").createOrReplaceTempView('total_retention_users')

spark.sql("select * from total_retention_users limit 10").show(10, False)


In [0]:

sum(test_retention.estimate*test_usage.est_average_active_users) as total_retention_users, sum(test_usage.est_average_active_users) as sum_au
domain_retention_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.retention-days.v6/fact/granularity=monthly/"
domain_basic_path = "s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.basic.v4/fact/granularity=w/date=2020-05-16"


In [0]:

spark.sql("""select sum(test_retention.estimate*test_usage.est_average_active_users) as total_retention_users, sum(test_usage.est_average_active_users) as sum_au from test_retention  
left join test_usage 
on test_retention.app_id = test_usage.app_id 
and test_retention.date = test_usage.date 
where test_retention.app_id = 20600004251992 
and test_retention.date between '2019-02-01' and '2019-08-31' 
and test_retention.kpi=301 
and test_usage.country_code='US' and test_retention.store_id=10 """).createOrReplaceTempView('total_retention_users')